In [12]:
import os
import sys

sys.path.insert(1, os.path.join(sys.path[0], ".."))

import numpy as np
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import torch

import data_utils.utils as data_utils
import inference.utils as inference_utils
import BigGAN_PyTorch.utils as biggan_utils
from data_utils.datasets_common import pil_loader
import torchvision.transforms as transforms
import time
from glob import glob

import torch
import torchvision
from glob import glob
import utils
import os
from PIL import Image
from data_utils.resnet import resnet50
import torch.nn as nn
import timm
from metrics import metric_utils
import torchvision.transforms as T
import torch.nn.functional as F

In [13]:
device = "cuda"

In [14]:
norm_mean = torch.Tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to("cuda")
norm_std = torch.Tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to("cuda")

In [15]:
net = utils.load_pretrained_feature_extractor(
        '/Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/swav_800ep_pretrain.pth.tar',
        "selfsupervised",
        "resnet50",
    )
# net = torch.nn.DataParallel(net)
net.to("cuda")
net.eval()

using resnet50 to extract features
Loading pretrained weights from:  /Work2/Watch_This/ICGAN/ic_gan/pretrained_models_path/swav_800ep_pretrain.pth.tar
key  projection_head.0.weight  not in dict
key  projection_head.0.bias  not in dict
key  projection_head.1.weight  not in dict
key  projection_head.1.bias  not in dict
key  projection_head.1.running_mean  not in dict
key  projection_head.1.running_var  not in dict
key  projection_head.1.num_batches_tracked  not in dict
key  projection_head.3.weight  not in dict
key  projection_head.3.bias  not in dict
key  prototypes.weight  not in dict
Network key  fc.weight  not in dict to load
Network key  fc.bias  not in dict to load


ResNet_mine(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride

In [35]:
dataset = utils.get_dataset_images(
        256,
        data_path="/Work1/imagenet",
        longtail=False,
        split="train",
        test_part=False,
        which_dataset="imagenet",
        instance_json="",
        stuff_json="",
    )

Generating  Index file imagenet_imgs.npz...


100%|█████████████████████████████████████████████| 2/2 [00:01<00:00,  1.00it/s]


In [36]:
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    num_workers=62,
    pin_memory=True,
#     sampler=sampler,
)

In [37]:
#Saving Encodings

gpu = torch.device("cuda")
for ind, data2 in enumerate(loader):
    with torch.no_grad():
        start_time = time.time()
        data = data2[0].cuda(gpu, non_blocking=True)
        x_tf1 = data * 0.5 + 0.5
        x_tf2 = (x_tf1 - norm_mean) / norm_std
        x_tf3 = torch.nn.functional.upsample(x_tf2, 224, mode="bicubic")
        x_feat1, _ = net(x_tf3)
        x_feat2 =  x_feat1/torch.linalg.norm(x_feat1, dim=1, keepdims=True)
        x_feat3 = x_feat2.detach().cpu().numpy()
        
        for ind2, file_name in enumerate(data2[3]):
            np.save(file_name.split(".")[0] + ".npy", x_feat3[ind2])
    if ind % 1 == 0:
        print((ind/len(loader))*100.0)
    

here
0.0
here
0.07692307692307693
here
0.15384615384615385
here
0.23076923076923078
here
0.3076923076923077
here
0.38461538461538464
here
0.46153846153846156
here
0.5384615384615384
here
0.6153846153846154
here
0.6923076923076923
here
0.7692307692307693
here
0.8461538461538461
here
0.9230769230769231
here
1.0
here
1.0769230769230769
here
1.153846153846154
here
1.2307692307692308
here
1.3076923076923077
here
1.3846153846153846
here
1.4615384615384615
here
1.5384615384615385
here
1.6153846153846154
here
1.6923076923076923
here
1.7692307692307692
here
1.8461538461538463
here
1.9230769230769231
here
2.0
here
2.076923076923077
here
2.1538461538461537
here
2.230769230769231
here
2.307692307692308
here
2.3846153846153846
here
2.4615384615384617
here
2.5384615384615383
here
2.6153846153846154
here
2.6923076923076925
here
2.769230769230769
here
2.8461538461538463
here
2.923076923076923
here
3.0
here
3.076923076923077
here
3.1538461538461537
here
3.230769230769231
here
3.307692307692308
here
3.3